In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from time import sleep
import threading
import random

In [2]:
class SeleniumScraper:
    def __init__(self):
        self.driver = self.setup_webdriver()
        self.wait_time = random.random()*4+2

    def setup_webdriver(self):
        my_options = webdriver.ChromeOptions()
        my_options.add_argument("--incognito")
        my_service = Service()
        driver = webdriver.Chrome(
            options=my_options,
            service=my_service
        )
        return driver

    def open_website(self):
        self.driver.get("https://portal.sw.nat.gov.tw/APGQ/XGB301")  #關港貿單一窗口
        sleep(self.wait_time)

    def input_declaration_number(self, cpno):
        inputElement = self.driver.find_element(By.ID, "declNo")  #報單號碼欄位id
        inputElement.send_keys(cpno)
        inputElement.send_keys(Keys.RETURN)
        sleep(self.wait_time)

    def output_results(self):
        outputElement = self.driver.find_element(By.ID, "detail_grid2")  #查詢結果欄
        results = outputElement.text.split("\n")

        for i in range(len(results)-1, 0, -1):
            if "放行" in results[i]:
                print(results[i])
        print("=" * 50)

    def close_driver(self):
        self.driver.quit()

In [3]:
# from selenium_scraper import SeleniumScraper  # 假設你的爬蟲程式放在selenium_scraper.py中

def crawl(query_target):
    scraper = SeleniumScraper()
    scraper.open_website()
    scraper.input_declaration_number(query_target)
    print("報單號碼:",query_target)
    scraper.output_results()
    scraper.close_driver()

if __name__ == "__main__":
    
    query_targets = []
    with open("./報單號碼查詢.txt", "r") as file:
        query_targets += [line.strip() for line in file.readlines()]   
#     query_targets = ["AAF212557A0181","CA  1213300833","CA  1213300832"]  # 這裡替換成你的查詢目標列表

    # 創建一個多執行緒列表
    threads = []

    # 創建並啟動多執行緒
    for query_target in query_targets:
        thread = threading.Thread(target=crawl, args=(query_target,))
        threads.append(thread)
        thread.start()

    # 等待所有多執行緒完成
    for thread in threads:
        thread.join()


報單號碼: AAF212557A0181
14 112-08-16 10:13:55 RL 放行 N0000-8G173002
報單號碼: CA  1213300833
11 112-07-18 14:22:16 RL 放行 N0000-7I063305
報單號碼: CA  1213300832
